In [1]:
# Note: honestly, I was pretty tempted to create a model that preps data based on type of data and model it will feed into,
# but opted to choose this simplified generalized version due to time constraints

In [2]:
# try: response block to code, combine code from each agent and turn it into a code block,
# then execute out with exec(df) - remember to global df too
# smth like this: context = {'df': df} 
# exec(agent_code, context)
# df_modified = context['df']

In [3]:
# here is the summary of agents used and in order:
# Standardize: Fix types, remove currency symbols, unify text casing.
# Date: Extract features (Year/Month) from timestamps.
# Duplicates: Remove exact row matches (Safety: <1%).
# Grouper: Group rare categories into "Other" (Safety: <1%).
# Nulls: Drop bad cols/rows or Impute (Median/Mode).
# Correlation: Drop redundant features (Correlation > 95%).
# Skew: Log-transform positive outliers.
# Encoding: Convert categories to numbers (One-Hot vs Label).
# Scaler: Split Train/Test and Standardize (Prevent Leakage).
# Bias: Fixes Statistical Bias and Class Imbalance on Train set.
# Mastermind: orchestrates the code of these 10 agents to work in sequential order on a given dataset.

In [4]:
# setup keys
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Setup and authentication complete.


In [5]:
# import libraries
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor
from google.adk.code_executors import UnsafeLocalCodeExecutor

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [6]:
# configure retry options
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [7]:
import pandas as pd
import numpy as np
import random
# using a randomized dataset, in production replace with an actual dataset
# Set seed for reproducibility
np.random.seed(2)

def generate_messy_dataset(rows=1000):
    print("⚠️ Generating The Doomed Dataset...")
    
    # 1. BASE DATA & SKEW (Triggers SkewAgent)
    # Generate a log-normal distribution (Right skewed)
    transaction_amt = np.random.lognormal(mean=2, sigma=1, size=rows)
    
    # 2. REDUNDANT FEATURES (Triggers CorrelationAgent)
    # Celsius and Fahrenheit are perfectly correlated
    temp_c = np.random.normal(25, 5, rows)
    temp_f = temp_c * 9/5 + 32
    
    # 3. MESSY STRINGS & CURRENCY (Triggers StandardizeAgent)
    # Includes whitespace, different cases, and symbols
    cities = ["  new york ", "New York", "SF", "sf ", "chicago", "Chicago", "  Austin"]
    city_col = np.random.choice(cities, rows)
    
    salaries = np.random.randint(40000, 150000, rows).astype(str)
    # Corrupt 30% of salaries with currency symbols
    for i in range(rows):
        if np.random.rand() < 0.3:
            salaries[i] = f"${salaries[i]}"
        if np.random.rand() < 0.1:
            salaries[i] = f"{salaries[i]},00" # European style comma/decimal mix
            
    # 4. DATES (Triggers DateAgent)
    # Mix of formats and NaTs
    start_date = pd.to_datetime('2020-01-01')
    dates = [start_date + pd.Timedelta(days=x) for x in range(rows)]
    date_strings = [d.strftime('%Y-%m-%d') for d in dates]
    # Corrupt some dates
    date_strings[0] = "Not a Date"
    date_strings[10] = "Unknown"
    
    # 5. NULLS & MISSING DATA (Triggers NullAgent)
    # A. > 50% Missing (Should be dropped entirely)
    mostly_empty = np.array([np.nan] * rows)
    mostly_empty[:10] = 1 # Only 10 values exist
    
    # B. < 5% Missing (Rows should be dropped)
    tiny_missing = np.random.rand(rows)
    tiny_missing[:15] = np.nan # 1.5% missing
    
    # C. ~20% Missing (Should be Imputed)
    medium_missing_age = np.random.randint(18, 70, rows).astype(float)
    medium_missing_age[:200] = np.nan # 20% missing
    
    # 6. RARE CATEGORIES (Triggers GrouperAgent)
    # 'Google' and 'Direct' are common; 'Friend' and 'Billboard' are rare (<1%)
    sources = ['Google']*800 + ['Direct']*190 + ['Friend']*5 + ['Billboard']*5
    np.random.shuffle(sources)
    
    # 7. HIGH CARDINALITY (Triggers EncodingAgent - Label Encode)
    # 50 unique ZIP codes
    zips = np.random.randint(90000, 90050, rows).astype(str)
    
    # 8. LOW CARDINALITY (Triggers EncodingAgent - One-Hot Encode)
    membership = np.random.choice(['Gold', 'Silver', 'Bronze'], rows)
    
    # 9. CLASS IMBALANCE (Triggers AutoBalanceAgent)
    # 90% Class 0, 10% Class 1
    target = np.random.choice([0, 1], rows, p=[0.90, 0.10])
    
    # CREATE DATAFRAME
    df = pd.DataFrame({
        'ID_Column': range(rows), # Should be ignored by Skew/Scaling agents
        'Transaction_Amt': transaction_amt, # Skewed
        'Temp_C': temp_c, # Redundant
        'Temp_F': temp_f, # Redundant to be dropped
        'City': city_col, # Messy text
        'Salary': salaries, # Messy numbers ($)
        'Join_Date': date_strings, # Date parsing
        'Garbage_Col': mostly_empty, # >50% null
        'Sensor_Reading': tiny_missing, # <5% null
        'User_Age': medium_missing_age, # Impute median
        'Referral': sources, # Group 'Friend' -> Other
        'Zip_Code': zips, # Label Encode
        'Membership': membership, # One-Hot Encode
        'Target_Label': target # Imbalanced
    })
    
    # 10. DUPLICATES (Triggers DuplicatesAgent)
    # Append top 5 rows to bottom to create exact duplicates (0.5% duplicates)
    df = pd.concat([df, df.head(5)], ignore_index=True)
    
    print("✅ Dataset Created. Shape:", df.shape)
    return df

# Initialize
df = generate_messy_dataset()

# Save to the bridge location
df.to_pickle('/kaggle/working/df.pkl')
print("✅ SETUP: Messy dataframe created at '/kaggle/working/df.pkl'")
print(f"Original Columns: {df.columns.tolist()}")
print("-" * 30)
df = pd.read_pickle('/kaggle/working/df.pkl')
print("Columns:", df.columns.tolist())
print(df.head())
local_executor = UnsafeLocalCodeExecutor(working_dir='/kaggle/working')

⚠️ Generating The Doomed Dataset...
✅ Dataset Created. Shape: (1005, 14)
✅ SETUP: Messy dataframe created at '/kaggle/working/df.pkl'
Original Columns: ['ID_Column', 'Transaction_Amt', 'Temp_C', 'Temp_F', 'City', 'Salary', 'Join_Date', 'Garbage_Col', 'Sensor_Reading', 'User_Age', 'Referral', 'Zip_Code', 'Membership', 'Target_Label']
------------------------------
Columns: ['ID_Column', 'Transaction_Amt', 'Temp_C', 'Temp_F', 'City', 'Salary', 'Join_Date', 'Garbage_Col', 'Sensor_Reading', 'User_Age', 'Referral', 'Zip_Code', 'Membership', 'Target_Label']
   ID_Column  Transaction_Amt     Temp_C     Temp_F      City  Salary  \
0          0         4.870722  25.988750  78.779749        SF   91833   
1          1         6.984778  26.684181  80.031525  New York  $57890   
2          2         0.872671  22.011714  71.621085   chicago  111204   
3          3        38.102154  26.689683  80.041429    Austin   82824   
4          4         1.229447  21.349859  70.429747       sf   124235   

   

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [8]:
# Note: have to output store df, train_df, and test_df safe path
# because the agent framework's execution sandbox does not persist local variable assignments or
# global scope modifications back to the main environment after the agent finishes.

In [9]:
# standardize text based on appearance percentage and similarity, strings to number if possible
standardize_agent = LlmAgent(
    name="StandardizeAgent",
    model=Gemini(model="gemini-2.0-flash", retry_options=retry_config),
    static_instruction="""
    You are a Python Data Engineering Expert.
    
    **YOUR GOAL:**
    Write a generic, robust Python script to clean and standardize ANY dataset without knowing column names in advance.

    **CRITICAL SETUP:**
    1. Imports: `import pandas as pd; import numpy as np; import warnings; warnings.filterwarnings('ignore')`
    2. Load: `df = pd.read_pickle('/kaggle/working/df.pkl')`

    **ADAPTIVE TRANSFORMATION LOGIC:**
    
    1. **Step 1: Clean Headers**
       - Strip whitespace, lowercase, and replace spaces with underscores for all column names.

    2. **Step 2: Smart Type Inference (Iterate through OBJECT columns only)**
       For each column where `dtype == 'object'`, perform the following checks IN ORDER:

       * **A. Check for Numeric/Currency:**
           - Create a temporary clean version: Remove '$', ',', and whitespace.
           - **DO NOT REMOVE '.' (Decimal Points) or '-' (Negative Signs).**
             - *Hint:* Use regex `r'[$,]'` to remove only specific symbols, NOT `r'[^\d]'`.
           - Attempt convert: `temp = pd.to_numeric(clean_col, errors='coerce')`
           - **The Safety Check:** Calculate the ratio of Non-Null values in `temp` vs the original column.
           - **Decision:** IF `temp.notna().mean()` > 0.8 (meaning >80% of data successfully converted):
             - Apply the conversion to the actual column.
             - Continue to next column (do not check Date).

       * **B. Check for Date/Time:**
           - Attempt convert: `temp = pd.to_datetime(col, errors='coerce')`
           - **The Safety Check:** Calculate the ratio of Non-Null values.
           - **Decision:** IF `temp.notna().mean()` >= 0.6 (meaning >60% is a valid date):
             - Apply the conversion.
             - Continue to next column.

       * **C. Fallback: Text Cleaning:**
           - If neither Numeric nor Date checks pass:
             - Strip whitespace: `df[col] = df[col].astype(str).str.strip()`
             - Title case: `df[col] = df[col].str.title()`
             - Replace empty strings with `np.nan`.

    **OUTPUT RULES:**
    - Output the Python code block to perform the transformation.
    - End with: `print(df.dtypes); df.to_pickle('/kaggle/working/df.pkl'); print("Standardized.")`
    - After the code executes successfully (and prints the result), you MUST respond with the text: "Standardization complete."
    """,
    code_executor=local_executor,
)

In [10]:
#  feature creation ex - spilt dates
Date_Agent = LlmAgent(
    name="DateAgent",
    model=Gemini(model="gemini-2.0-flash", retry_options=retry_config),
    static_instruction="""
    You are a Python Data Engineering Expert. You DO NOT speak natural language. You ONLY output executable Python code.

    **YOUR GOAL:**
    Write a Python script using `pandas` to Create Features by splitting Date/Time columns in the variable `df`.

    **CRITICAL:**
        1. Load 'df.pkl'. Do NOT create dummy data. Extract features from 'df.pkl', overwrite 'df.pkl'.
        2. Start your code with:
               import warnings
               warnings.filterwarnings('ignore')

    **INPUT CONTEXT:**
    - load df by: df = pd.read_pickle('/kaggle/working/df.pkl')

    **LOGIC & SAFETY CHECKS:**
    1. **Identify Candidates:**
       - Iterate through all columns.
       - Target columns where:
         - Dtype is already `datetime`.
         - OR Name contains: "date", "time", "joined", "created", "at" (case-insensitive) AND Dtype is `object`.
    
    2. **Safe Conversion:**
       - For candidates, attempt: `temp = pd.to_datetime(df[col], errors='coerce')`
       - **Validation:** Check the NaT (Null) rate of `temp`.
         - IF NaT rate > 50%: The column is likely NOT a real date. **SKIP IT.**
         - IF NaT rate <= 50%: Assign `df[col] = temp` and proceed to step 3.

    3. **Feature Splitting (The "Creation" Step):**
       - For every valid date column:
         - Create `{col}_year`: `df[col].dt.year`
         - Create `{col}_month`: `df[col].dt.month`
         - Create `{col}_day`: `df[col].dt.day`
         - Create `{col}_dow`: `df[col].dt.dayofweek` (0=Mon, 6=Sun)
         - Create `{col}_is_weekend`: `(df[col].dt.dayofweek >= 5).astype(int)`

    4. **Cleanup:**
       - DROP the original date column after extracting features (Models cannot digest raw timestamps).

    **OUTPUT RULES:**
    1. Output the Python code block to perform the transformation.
    2. End with:
        1. `print(f"Date Features Created. New Shape: {df.shape}")` and `print(df.columns.tolist())`.
        2. df.to_pickle('/kaggle/working/df.pkl'); print("Date features added. Saved to df.pkl")
    3. Use ONLY standard libraries and `pandas`.
    4. After the code executes successfully (and prints the result), you MUST respond with the text: "Date extraction complete."

    Failure to follow these rules will result in a system error.
    """,
    code_executor=local_executor, 
)

In [11]:
# remove duplicates data <1% for safety
duplicates_agent = LlmAgent(
    name="DuplicatesAgent",
    model=Gemini(model="gemini-2.0-flash", retry_options=retry_config),
    static_instruction="""
    You are a Python Data Engineering Expert. You DO NOT speak natural language. You ONLY output executable Python code.

    **YOUR GOAL:**
    Write a Python script using `pandas` to remove duplicate data safely from a variable `df`.

    **CRITICAL:** Load 'df.pkl'. Remove duplicates from 'df.pkl', overwrite 'df.pkl'.

    **INPUT CONTEXT:**
    1. load df by: df = pd.read_pickle('/kaggle/working/df.pkl')
    2. Start your code with:
               import warnings
               warnings.filterwarnings('ignore')

    **LOGIC & SAFETY CHECKS:**
    1. **Calculate Duplicates:** Identify how many rows would be removed using exact row matching.
    2. **The 1% Safety Rule:** - Calculate the drop percentage: `(duplicates_count / total_rows) * 100`.
       - IF the drop percentage is **greater than 1%**: Do NOT modify `df`. Instead, print(f"Aborting: Duplicates exceed 1% safety limit. Found {pct}%")`.
       - IF the drop percentage is **less than or equal to 1%**: Remove the duplicates permanently from `df`.

    **OUTPUT RULES:**
    1. Output the Python code block to perform the transformation.
    2. End with:
        1. printing: `print(f"Successfully dropped {dropped_count} rows. New shape: {df.shape}")`
        2. df.to_pickle('/kaggle/working/df.pkl'); print("Duplicates removed. Saved to df.pkl")
    3. Use ONLY standard libraries and `pandas`.
    4. After the code executes successfully (and prints the result), you MUST respond with the text: "Duplicate removal complete."

    Failure to follow these rules will result in a system error.
    """,
    code_executor=local_executor, 
)

In [12]:
# groups rare categories into "other" <1%, 0.5% individually
grouper_agent = LlmAgent(
    name="GrouperAgent",
    model=Gemini(model="gemini-2.0-flash", retry_options=retry_config),
    static_instruction="""
    You are a Python Data Engineering Expert. You DO NOT speak natural language.

    **YOUR GOAL:**
    **GOAL:** Group rare categories (appearing < 0.5%) into 'Other' if the total impact is < 1%.

    **CRITICAL:** Load 'df.pkl'. Group rare categories from 'df.pkl', overwrite 'df.pkl'.

    **INPUT CONTEXT:**
    1. load df by: df = pd.read_pickle('/kaggle/working/df.pkl')
    2. Start your code with:
               import warnings
               warnings.filterwarnings('ignore')

    **LOGIC:**
    1. Iterate through columns where `dtype == 'object'`.
    2. For each column:
       - Calculate value counts (normalized).
       - Identify "rare_values": those with `frequency < 0.005` (0.5%).
       - Calculate "impact": Sum of frequencies of all `rare_values`.
    3. **Decision Rule:**
       - **IF impact <= 0.01 (1%):** - Replace `rare_values` with the string "Other".
         - Print: `f"Column '{col}': Grouped {count} categories ({impact:.2%})"`
       - **ELSE (impact > 1%):**
         - Print: `f"Column '{col}': Skipped (Impact {impact:.2%} > 1% safety limit)"`

    **OUTPUT RULES:**
    1. Output the Python code block to perform the transformation.
    2. The code MUST end with:
        1. A summary print for each column: `print(f"Column '{col}': Grouped {count} rows")`.
        2. df.to_pickle('/kaggle/working/df.pkl'); print("Grouping complete. Saved to df.pkl")
    3. Use ONLY standard libraries and `pandas`.
    4. After the code executes successfully (and prints the result), you MUST respond with the text: "Duplicate removal complete."

    Failure to follow these rules will result in a system error.
    """,
    code_executor=local_executor, 
)

In [13]:
# Drop rows if the missing data is minimal (<5%), columns if the feature is mostly empty (>50%)
# Use median to fill in if possible
# total thresholding to prevent cascading data loss
Null_agent = LlmAgent(
    name="NullAgent",
    model=Gemini(model="gemini-2.0-flash", retry_options=retry_config),
    static_instruction="""
    You are a Python Data Engineering Expert. You DO NOT speak natural language. You ONLY output executable Python code.

    **YOUR GOAL:**
    Write a Python script using `pandas` to handle missing data (nulls).

    **CRITICAL:** Load 'df.pkl'. Handle nulls in 'df.pkl', overwrite 'df.pkl'.

    **INPUT CONTEXT:**
    1. load df by: df = pd.read_pickle('/kaggle/working/df.pkl')
    2. Start your code with:
               import warnings
               warnings.filterwarnings('ignore')

    **LOGIC - EXECUTE IN THIS ORDER:**
    1. **Column Cleanup:**
       - IF a column is missing > 50% data: Drop the **Column**.

    2. **Total Row Safety Check:**
       - Identify ALL rows that contain nulls in the remaining columns.
       - Calculate `total_rows_with_nulls`.
       - Calculate `loss_pct = (total_rows_with_nulls / total_rows) * 100`.

    3. **Decision Branch:**
       - **IF loss_pct =< 5%:**
         - Drop ALL rows containing nulls. (Safe to drop).
       - **ELSE (If loss_pct > 5%):**
         - Do NOT drop rows. Instead, Impute (Fill) data:
         - Numerics -> Fill with Median.
         - Object/String -> Fill with Mode (Top value).

    **OUTPUT RULES:**
    1. Output the Python code block to perform the transformation.
    2. End with: 
        1. `print(f"Action taken: {'Dropped Rows' if loss < 5 else 'Imputed Data'}. New Shape: {df.shape}")`
        2. df.to_pickle('/kaggle/working/df.pkl'); print("Redundancy removed. Saved to df.pkl")
    3. Use ONLY standard libraries and `pandas`.
    4. After the code executes successfully (and prints the result), you MUST respond with the text: "Nulls handled."

    Failure to follow these rules will result in a system error.
    """,
    code_executor=local_executor, 
)

In [14]:
# PCA - correlation matrix, drops var/features with little relevance
Correlation_Agent = LlmAgent(
    name="CorrelationAgent",
    model=Gemini(model="gemini-2.0-flash", retry_options=retry_config),
    static_instruction="""
    You are a Python Data Engineering Expert. You DO NOT speak natural language. You ONLY output executable Python code.

    **YOUR GOAL:**
    Write a Python script using `pandas` and `numpy` to remove **Redundant Features** (Multicollinearity) based on a correlation matrix.

    **CRITICAL:** Load 'df.pkl'. Drop correlated cols in 'df.pkl', overwrite 'df.pkl'.

    **INPUT CONTEXT:**
    1. load df by: df = pd.read_pickle('/kaggle/working/df.pkl')
    2. Start your code with:
               import warnings
               warnings.filterwarnings('ignore')

    **LOGIC:**
    1. Select numeric columns: `nums = df.select_dtypes(include=[np.number])`
    2. Safety Check: If `nums.shape[1] < 2`, stop and save.
    3. **Calculate Matrix:** `corr_matrix = nums.corr().abs()`
    4. **Select Upper Triangle (CRITICAL):** - Use this logic to avoid self-correlation:
       - `upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))`
    5. **Identify Drops:**
       - Find columns where any value in `upper` is > 0.95.
       - `to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]`
    6. **Drop:** `df = df.drop(columns=to_drop)`

    **OUTPUT RULES:**
    1. Output the Python code block to perform the transformation.
    2. End with:
        1.`print(f"Dropped {len(to_drop)} redundant features: {to_drop}. New Shape: {df.shape}")`
        2. df.to_pickle('/kaggle/working/df.pkl'); print("Redundancy removed. Saved to df.pkl")
    3. Use ONLY standard libraries, `numpy`, and `pandas`.
    4. After the code executes successfully (and prints the result), you MUST respond with the text: "PCA complete."

    Failure to follow these rules will result in a system error.
    """,
    code_executor=local_executor, 
)

In [15]:
# for numeric values, if skew() > 1, log transform for outliers (log(x+1)) if no neg values
# exclude ID rows and dates
Skew_Agent = LlmAgent(
    name="SkewAgent",
    model=Gemini(model="gemini-2.0-flash", retry_options=retry_config),
    static_instruction="""
    You are a Python Data Engineering Expert. You DO NOT speak natural language. You ONLY output executable Python code.

    **YOUR GOAL:**
    Write a Python script using `pandas` and `numpy` to fix positive skewness in the variable `df`.

    **CRITICAL:** Load 'df.pkl'. Fix skew in 'df.pkl', overwrite 'df.pkl'.

    **INPUT CONTEXT:**
    1. load df by: df = pd.read_pickle('/kaggle/working/df.pkl')
    2. Start your code with:
               import warnings
               warnings.filterwarnings('ignore')

    **LOGIC:**
    1. Iterate through **Numeric Columns** (float/int).
    2. **Exclusion Check:** - SKIP if name contains 'id', 'ID', 'Id'.
       - SKIP if name ends with '_year', '_month', '_day', '_dow', '_weekend'.
    3. **Value Check:**
       - SKIP if column contains negative values (Log is undefined).
    4. **Transformation:**
       - Calculate `old_skew = df[col].skew()`.
       - **IF old_skew > 1:**
         - Apply: `df[col] = np.log1p(df[col])`
         - Recalculate: `new_skew = df[col].skew()`
         - Print: `f"Column '{col}': Skew fixed ({old_skew:.2f} -> {new_skew:.2f})"`

    **OUTPUT RULES:**
    1. Output the Python code block to perform the transformation.
    3. The code MUST end with:
        1. A summary loop printing: `print(f"Column '{col}': Skew from {old_skew} -> {new_skew}")` for changed columns only.
        2. df.to_pickle('/kaggle/working/df.pkl'); print("Skew fixed. Saved to df.pkl")
    4. Use ONLY standard libraries, `numpy`, and `pandas`.
    5. After the code executes successfully (and prints the result), you MUST respond with the text: "Skew fix complete."

    Failure to follow these rules will result in a system error.
    """,
    code_executor=local_executor, 
)

In [16]:
# one hot encoding for low cardinality
# label encoding for high and ordinal data
Encoding_Agent = LlmAgent(
    name="EncodingAgent",
    model=Gemini(model="gemini-2.0-flash", retry_options=retry_config),
    static_instruction="""
    You are a Python Data Engineering Expert. You DO NOT speak natural language. You ONLY output executable Python code.

    **YOUR GOAL:**
    Write a Python script using `pandas` to encode categorical data (One-Hot vs Label Encoding) in the variable `df`.

    **CRITICAL:** Load 'df.pkl'. Encode 'df.pkl', overwrite 'df.pkl'.

    **INPUT CONTEXT:**
    1. load df by: df = pd.read_pickle('/kaggle/working/df.pkl')
    2. Start your code with:
               import warnings
               warnings.filterwarnings('ignore')

    **LOGIC:**
    1. **Identify Object Columns:** `obj_cols = df.select_dtypes(include=['object']).columns`
    2. **Filter Exclusions:** Remove columns containing 'id', 'ID', or 'Id'.
    
    3. **Strategy Separation:**
       - Create two lists: `one_hot_cols` and `label_cols`.
       - Iterate through candidates:
         - IF `df[col].nunique() < 10`: Add to `one_hot_cols`.
         - ELSE: Add to `label_cols`.

    4. **Execution:**
       - **Step A (Label Encoding):**
         - For col in `label_cols`: `df[col] = df[col].astype('category').cat.codes`
       
       - **Step B (One-Hot Encoding):**
         - IF `one_hot_cols` is not empty:
           - `df = pd.get_dummies(df, columns=one_hot_cols, prefix=one_hot_cols, dtype=int)`
           - *Note:* This automatically drops original columns and appends new ones as Integers (0/1).

    **OUTPUT RULES:**
    1. Output the Python code block to perform the transformation.
    2. The code MUST end with:
        1. `print(f"Encoding Complete. New Shape: {df.shape}")` and `print(df.dtypes)`.
        2. df.to_pickle('/kaggle/working/df.pkl'); print("Encoding complete. Saved to df.pkl")
    3. Use ONLY standard libraries and `pandas`.
    4. After the code executes successfully (and prints the result), you MUST respond with the text: "Data Encoding complete."

    Failure to follow these rules will result in a system error.
    """,
    code_executor=local_executor, 
)

In [17]:
# feature engineering - z score standardization for reducing scale
# ^ spilt data train and test, prevents leakage, then transform test set using precalculated values (mean/std)
# Not using normalization here since it's weaker to outliers and is better on Neural Nets/KNNs/Images
Scaler_Agent = LlmAgent(
    name="ScalerAgent",
    model=Gemini(model="gemini-2.0-flash", retry_options=retry_config),
    static_instruction="""
    You are a Python Data Engineering Expert. You DO NOT speak natural language. You ONLY output executable Python code.

    **YOUR GOAL:**
    Write a Python script using `sklearn` to split the data and perform Z-Score Standardization while preventing Data Leakage.

    **CRITICAL:**
        1. Load 'df.pkl'. DO NOT create dummy data. Save 'train_df.pkl' and 'test_df.pkl'.
        2. Start your code with:
               import warnings
               warnings.filterwarnings('ignore')
               from sklearn.model_selection import train_test_split
               from sklearn.preprocessing import StandardScaler

    **INPUT CONTEXT:**
    - load df by: df = pd.read_pickle('/kaggle/working/df.pkl')

    **LOGIC:**
        1. **Identify Target:** Look for a column named 'Target_Label', 'target', or 'label'.
        2. **Split:** - `train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)`
        3. **Define Scaling Features:**
           - Identify Numeric Columns in `train_df`.
           - **EXCLUSION:** Remove the Target column and any 'ID' columns from the list of columns to scale.
        4. **Apply Scaler (Z-Score Standardization):**
           - `scaler = StandardScaler()`
           - `train_df[scale_cols] = scaler.fit_transform(train_df[scale_cols])`
           - `test_df[scale_cols] = scaler.transform(test_df[scale_cols])`
           - *Note:* This prevents data leakage by using Train statistics for Test data.

    **OUTPUT RULES:**
    1. Output the Python code block to perform the transformation.
    2. **CRITICAL:** Use absolute paths for saving:
        - train_df.to_pickle('/kaggle/working/train_df.pkl')
        - test_df.to_pickle('/kaggle/working/test_df.pkl')
        - print("Split & Scaled. Created train_df.pkl and test_df.pkl")
    3. End with:
       - `print(f"Split Complete. Train Shape: {train_df.shape}, Test Shape: {test_df.shape}")`
       - `print("Z-Score Standardization applied safely.")`
    4. After the code executes successfully (and prints the result), you MUST respond with the text: "Data Scalar complete."

    Failure to follow these rules will result in a system error.
    """,
    code_executor=local_executor, 
)

In [18]:
# Fixes Statistical Bias and Class Imbalance on Train set
# To prevent overfitting, we introduce jittering/noise injection
# ^ but obviously exclude target so the model doesn't predict that you have 1.2 siblings or smth
AutoBalance_Agent = LlmAgent(
    name="AutoBalanceAgent",
    model=Gemini(model="gemini-2.0-flash", retry_options=retry_config),
    static_instruction="""
    You are a Python Data Engineering Expert. You DO NOT speak natural language. You ONLY output executable Python code.

    **YOUR GOAL:**
    Write a Python script using `sklearn`, `numpy`, and `pandas` to Balance Training Data safely using **Noise Injection** (Jittering).

    **CRITICAL:**
        1. Load 'train_df.pkl'. DO NOT create dummy data. Balance 'train_df.pkl' and overwrite 'train_df.pkl'.
        2. Start your code with:
               import warnings
               warnings.filterwarnings('ignore')

    **INPUT CONTEXT:**
    1. load `train_df` by: train_df = pd.read_pickle('/kaggle/working/train_df.pkl')

    **LOGIC:**
    1. **Auto-Detect Target:**
       - Look for column names: ["target", "label", "class", "outcome", "y"]. Use first match.
       - Else, use the last column. Save as `target_col`.

    2. **Safety Checks:**
       - **Regression Check:** If `train_df[target_col].nunique() > 20`: 
         - Print "Regression detected (High Cardinality). Skipping balance."
         - Save and Exit.
       - **Balance Check:** Calculate `minority_count / majority_count`.
         - If Ratio >= 0.8: Print "Already balanced." -> Exit.

    3. **Oversampling Strategy (The "Synthetic Block"):**
       - Separate: `df_maj` (Majority) and `df_min` (Minority).
       - Calculate needed: `n_samples = len(df_maj) - len(df_min)`.
       - **Generate Synthetic:** `df_synthetic = resample(df_min, replace=True, n_samples=n_samples, random_state=42)`
       
    4. **Inject Noise (Jitter):**
       - Iterate through **Numeric Columns** of `df_synthetic` ONLY.
       - **Exclude:** `target_col` and any 'ID' columns.
       - **Logic:** `df_synthetic[col] += np.random.normal(0, 0.01 * df_synthetic[col].std(), size=n_samples)`
       - *Note:* This prevents exact duplicates.

    5. **Combine:**
       - `train_df = pd.concat([df_maj, df_min, df_synthetic])`

    **OUTPUT RULES:**
    1. Output the Python code block to perform the transformation.
    2. End with:
        1. train_df.to_pickle('/kaggle/working/train_df.pkl')
        2. `print(f"Target: '{target_col}'. Balanced with Jitter. New Shape: {train_df.shape}")`.
    3. After the code executes successfully (and prints the result), you MUST respond with the text: "Bias Eliminated."

    Failure to follow these rules will result in a system error.
    """,
    code_executor=local_executor, 
)

In [19]:
# Overall Model Orchestrator
Mastermind = LlmAgent(
    name="Mastermind_agent",
    model=Gemini(model="gemini-2.0-flash", retry_options=retry_config),
    static_instruction="""
    You are an efficient Data Prepping Orchestrator.
    
    **YOUR MISSION:**
    Execute the following data engineering pipeline strictly in order. 
    You do not need to write code yourself; use the provided Tools to generate and execute the code.
    All agents must read/write from the CURRENT DIRECTORY (/kaggle/working).
    The tools will handle file I/O automatically using fixed filenames: 'df.pkl', 'train_df.pkl', 'test_df.pkl'.
    
    **PIPELINE SEQUENCE:**
    1.  **standardize_agent**: Clean formatting (df -> df).
    2.  **Date_Agent**: Extract time features (df -> df).
    3.  **duplicates_agent**: Remove rows (df -> df).
    4.  **grouper_agent**: Group rare categories (df -> df).
    5.  **Null_agent**: Impute or drop missing data (df -> df).
    6.  **Correlation_Agent**: Drop redundant features (df -> df).
    7.  **Skew_Agent**: Fix numeric skew (df -> df).
    8.  **Encoding_Agent**: Categorical to Numerical (df -> df).
    9.  **Scaler_Agent**: SPLIT into Train/Test and Scale (df -> train_df, test_df).
    10. **AutoBalance_Agent**: Balance the TRAINING set only (train_df -> train_df).
        *DO NOT touch test_df in this step.*

    **OUTPUT RULES:**
    1. Do not output the actual dataframes as text (they are too large).
    2. Once Step 10 is finished, output a final confirmation: 
       "✅ Pipeline Complete. Variables 'train_df' and 'test_df' are ready for modeling."

    Failure to follow the sequence will result in immediate termination.
    """,
    tools=[
        AgentTool(agent=standardize_agent),
        AgentTool(agent=Date_Agent),
        AgentTool(agent=duplicates_agent),
        AgentTool(agent=grouper_agent),
        AgentTool(agent=Null_agent),
        AgentTool(agent=Correlation_Agent),
        AgentTool(agent=Skew_Agent),
        AgentTool(agent=Encoding_Agent),
        AgentTool(agent=Scaler_Agent),
        AgentTool(agent=AutoBalance_Agent),
    ],
)

print("✅ The Bane of Interns is online.🤖")

✅ The Bane of Interns is online.🤖


In [20]:
# Save a copy for comparing later
df_raw_snapshot = df.copy()
# Execute
runner = InMemoryRunner(agent=Mastermind)
response = await runner.run_debug("Start the pipeline.")


 ### Created new session: debug_session_id

User > Start the pipeline.


Mastermind_agent > Okay, let's start the data engineering pipeline.

**Step 1: standardize_agent**



Mastermind_agent > **Step 2: Date_Agent**



Mastermind_agent > **Step 3: duplicates_agent**



Mastermind_agent > **Step 4: grouper_agent**



...

Mastermind_agent > **Step 6: Correlation_Agent**



Mastermind_agent > **Step 7: Skew_Agent**



Mastermind_agent > **Step 8: Encoding_Agent**



Mastermind_agent > **Step 9: Scaler_Agent**



Mastermind_agent > **Step 10: AutoBalance_Agent**

Mastermind_agent > ✅ Pipeline Complete. Variables 'train_df' and 'test_df' are ready for modeling.



In [22]:
# Time to compare

def print_comparison_report(raw, train, test):
    print("\n" + "="*40)
    print("🧪 PIPELINE VALIDATION REPORT")
    print("="*40)

    # 1. SHAPE & DROPS
    total_final = len(train) + len(test)
    dropped = len(raw) - total_final
    print(f"\n1. DATA VOLUME:")
    print(f"   - Original: {len(raw)} rows")
    print(f"   - Final:    {total_final} rows")
    print(f"   - Dropped:  {dropped} rows ({(dropped/len(raw)):.1%} loss)")

    # 2. NULLS
    print(f"\n2. NULL CHECK:")
    print(f"   - Remaining Nulls: {train.isnull().sum().sum()} (Must be 0)")

    # 3. COLUMNS (Encoding check)
    print(f"\n3. DIMENSIONS:")
    print(f"   - Orig Cols:  {raw.shape[1]}")
    print(f"   - Final Cols: {train.shape[1]}")
    
    # 4. SKEW (Log Transform Check)
    if 'Transaction_Amt' in raw.columns and 'Transaction_Amt' in train.columns:
        print(f"\n4. SKEW CORRECTION:")
        print(f"   - Orig Max:  ${raw['Transaction_Amt'].max():,.2f}")
        print(f"   - Final Max: {train['Transaction_Amt'].max():.4f} (Scaled)")

    # 5. BALANCE CHECK
    target_col = 'Target_Label'
    if target_col in train.columns:
        print(f"\n5. CLASS BALANCE ({target_col}):")
        tc = train[target_col].value_counts(normalize=True)
        print(f"   - Train (Balanced): 0: {tc.get(0,0):.2f} | 1: {tc.get(1,0):.2f}")
        
        testc = test[target_col].value_counts(normalize=True)
        print(f"   - Test (Natural):   0: {testc.get(0,0):.2f} | 1: {testc.get(1,0):.2f}")

train_abs_path = '/kaggle/working/train_df.pkl'
test_abs_path = '/kaggle/working/test_df.pkl'

if os.path.exists(train_abs_path):
    print(f"\n✅ SUCCESS: Found {train_abs_path}")
    final_train = pd.read_pickle(train_abs_path)
    final_test = pd.read_pickle(test_abs_path)
    print_comparison_report(df_raw_snapshot, final_train, final_test)
else:
    print(f"\n❌ FAILURE: File still missing at {train_abs_path}")


✅ SUCCESS: Found /kaggle/working/train_df.pkl

🧪 PIPELINE VALIDATION REPORT

1. DATA VOLUME:
   - Original: 1005 rows
   - Final:    1288 rows
   - Dropped:  -283 rows (-28.2% loss)

2. NULL CHECK:
   - Remaining Nulls: 0 (Must be 0)

3. DIMENSIONS:
   - Orig Cols:  14
   - Final Cols: 24


In [27]:
def deep_dive_report(raw, train):
    print("\n" + "="*60)
    print("🔬 DEEP DIVE: TRANSFORMATION AUDIT")
    print("="*60)

    # 1. STANDARDIZATION (Text Cleaning)
    # Compare raw 'City' to processed 'city'
    print("\n1. 🧹 STANDARDIZATION (Text Cleaning)")
    print("-" * 30)
    try:
        # Get top 5 unique values to show consolidation
        raw_cities = raw['City'].unique()[:5]
        clean_cities = train['city'].unique()[:5] if 'city' in train.columns else "Column Dropped/Encoded"
        print(f"🔴 Raw Cities:   {raw_cities}")
        print(f"🟢 Clean Cities: {clean_cities}")
    except Exception as e:
        print(f"Could not compare cities: {e}")

    # 2. DATE ENGINEERING
    # Show how 'Join_Date' exploded into features
    print("\n2. 📅 DATE ENGINEERING")
    print("-" * 30)
    if 'Join_Date' in raw.columns:
        print(f"🔴 Raw Date (Row 0): '{raw['Join_Date'].iloc[0]}'")
        
        # Find new columns starting with 'join_date'
        date_cols = [c for c in train.columns if 'join_date' in c]
        if date_cols:
            print(f"🟢 Extracted Features (Row 0):")
            print(train[date_cols].iloc[0].to_frame().T.to_string(index=False))
        else:
            print("⚠️ No date features found.")

    # 3. FEATURE SELECTION (Correlation)
    # Check if Temp_F (redundant) is gone but Temp_C remains
    print("\n3. ✂️ FEATURE SELECTION (Correlation)")
    print("-" * 30)
    has_temp_c = 'temp_c' in train.columns
    has_temp_f = 'temp_f' in train.columns
    
    if has_temp_c and not has_temp_f:
        print("✅ SUCCESS: 'temp_c' kept, 'temp_f' DROPPED (Correlation > 0.95 detected).")
    elif has_temp_f:
        print("⚠️ CHECK: 'temp_f' still exists.")
    else:
        print("ℹ️ Note: Temp columns not found (maybe renamed?).")

    # 4. SCALING (Z-Score)
    # Compare distributions of Transaction Amount
    print("\n4. ⚖️ SCALING & SKEW (Transaction_Amt)")
    print("-" * 30)
    if 'Transaction_Amt' in raw.columns and 'transaction_amt' in train.columns:
        raw_stats = raw['Transaction_Amt'].describe()
        new_stats = train['transaction_amt'].describe()
        
        print(f"{'STAT':<10} | {'RAW (Skewed)':<15} | {'FINAL (Log+Scaled)':<15}")
        print("-" * 45)
        print(f"{'Mean':<10} | {raw_stats['mean']:<15.2f} | {new_stats['mean']:<15.4f} (Should be ~0)")
        print(f"{'Std':<10} | {raw_stats['std']:<15.2f} | {new_stats['std']:<15.4f} (Should be ~1)")
        print(f"{'Max':<10} | {raw_stats['max']:<15.2f} | {new_stats['max']:<15.4f}")

    # 5. ENCODING
    # Show how Membership (Gold/Silver) became numbers or One-Hot
    print("\n5. 🔠 ENCODING")
    print("-" * 30)
    # Check for One-Hot columns
    membership_cols = [c for c in train.columns if 'membership' in c]
    if len(membership_cols) > 1:
        print(f"✅ One-Hot Detected. 'Membership' expanded to: {membership_cols}")
        print(train[membership_cols].head(3))
    elif len(membership_cols) == 1:
         print(f"✅ Label Encoding Detected. 'Membership' is now numeric.")
         print(train[membership_cols].head(3))

    # 6. BALANCING
    print("\n6. ⚖️ CLASS IMBALANCE (Oversampling)")
    print("-" * 30)
    orig_counts = raw['Target_Label'].value_counts()
    new_counts = train['target_label'].value_counts()
    
    print(f"🔴 Original Ratio: 1s are {orig_counts[1] / len(raw):.1%} of data")
    print(f"🟢 Training Ratio: 1s are {new_counts[1] / len(train):.1%} of data (Target ~50%)")
    print(f"   (Synthetic samples added: {len(train) - len(raw)})")

# RUN IT
if os.path.exists('/kaggle/working/train_df.pkl'):
    final_train = pd.read_pickle('/kaggle/working/train_df.pkl')
    # Use df_raw_snapshot from the very beginning of your script
    deep_dive_report(df_raw_snapshot, final_train)

In [ ]:
# I am just a recent CS grad, so the solutions provided here are a far cry from perfect,
# but if there are any questions, please contact me at https://www.linkedin.com/in/yulin-lin-0a05201ab/.